In [ ]:
from unsloth import FastLanguageModel
import os
import torch
max_seq_length = 2048 # Qualsiasi valore, dato che viene effettuato RoPE Scaling in automatico.
dtype = None # None per auto detection.
load_in_4bit = True # Carica il modello quantizzato in 4bit.
lora_rank = 64 # Larger rank = smarter, but slower

hf_token = os.environ.get("HF_TOKEN") or os.environ.get("HUGGINGFACE_HUB_TOKEN")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Alelcv27/Qwen2.5-7B-Math-CoT",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
    max_lora_rank = lora_rank,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/.pyenv/versions/uns2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.676 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.33s/it]


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split = "train")#.shuffle()
#dataset = dataset.shuffle().select(range(120000))
dataset = dataset.take(10000)

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # Specifica il nome del campo nel dataset in cui è archiviato il testo di input
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Numero di processi da utilizzare per la pre-elaborazione del dataset
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 4, # Numero di training sample per batch per device (GPU/CPU)
        gradient_accumulation_steps = 4, # Il numero di step per accumulare i gradienti prima di eseguire un backward pass.
        max_grad_norm=1.0,
        warmup_ratio=0.05, # Numero di step iniziali durante i quali il learning rate aumenta linearmente da 0 al valore impostato.
        num_train_epochs = 1,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10, # Con quale frequenza (in termini di step) devono essere stampati i log di training.
        optim = "adamw_8bit", # Optimizer utilizzato
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # Tipo di learning rate scheduler da utilizzare
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=28): 100%|██████████| 10000/10000 [00:03<00:00, 2787.90 examples/s]


In [5]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.676 GB.
6.725 GB of memory reserved.


In [6]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 625
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.881500
20,0.545100
30,0.547300
40,0.559900
50,0.573100
60,0.533800
70,0.511900
80,0.549300
90,0.533800
100,0.550300


In [7]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2563.8581 seconds used for training.
42.73 minutes used for training.
Peak reserved memory = 9.764 GB.
Peak reserved memory for training = 3.039 GB.
Peak reserved memory % of max memory = 41.24 %.
Peak reserved memory for training % of max memory = 12.836 %.


In [8]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Implement a algorithm in Python for sorting two large lists A and B. The algorithm should take the first element of list A and compare it with the first element of list B. If it is greater, then it should add the element to the result list and move the pointer of B to the next element. If the element of A is smaller, the pointer should move to the next element of A, and so on.", # instruction
        "A = [3, 6, 8, 10, 11] \nB = [2, 5, 7, 12]", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Implement a algorithm in Python for sorting two large lists A and B. The algorithm should take the first element of list A and compare it with the first element of list B. If it is greater, then it should add the element to the result list and move the pointer of B to the next element. If the element of A is smaller, the pointer should move to the next element of A, and so on.

### Input:
A = [3, 6, 8, 10, 11] 
B = [2, 5, 7, 12]

### Response:
def merge_sort(A, B):
    result = []
    i, j = 0, 0
    while i < len(A) and j < len(B):
        if A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            result.append(B[j])
            j += 1
    result += A[i:]
    result += B[j:]
    return result

A = [3, 6, 8, 10, 11]
b = [2, 5, 7, 12]
print(merge_sort(a, b))<|im_end|>


In [9]:
output_repo = "Alelcv27/Qwen2.5-7B-Math-Code-v2"

model.push_to_hub_merged(output_repo, tokenizer, save_method = "merged_16bit", token = token)
#model.push_to_hub(output_repo, token = token)
#tokenizer.push_to_hub(output_repo, token = token)

Processing Files (1 / 1): 100%|██████████| 11.4MB / 11.4MB, 5.71MB/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  


Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 4 files from cache to `Alelcv27/Qwen2.5-7B-Math-Code-v2`: 100%|██████████| 4/4 [00:08<00:00,  2.15s/it]


Successfully copied all 4 files from cache to `Alelcv27/Qwen2.5-7B-Math-Code-v2`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 105517.08it/s]
Processing Files (1 / 1): 100%|██████████| 4.88GB / 4.88GB,  539kB/s  t/s]
New Data Upload: 100%|██████████| 3.79GB / 3.79GB,  539kB/s  
Processing Files (1 / 1): 100%|██████████| 4.93GB / 4.93GB, 51.6kB/s  , 320.51s/it]
New Data Upload: 100%|██████████| 4.93GB / 4.93GB, 51.6kB/s  
Processing Files (1 / 1): 100%|██████████| 4.33GB / 4.33GB, 23.1kB/s  , 371.19s/it]
New Data Upload: 100%|██████████| 4.33GB / 4.33GB, 23.1kB/s  
Processing Files (1 / 1): 100%|██████████| 1.09GB / 1.09GB,  191MB/s  , 363.40s/it]
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [18:11<00:00, 272.78s/it]


Unsloth: Merge process complete. Saved to `/home/user/Progetti/Merging/qwen notebooks/Alelcv27/Qwen2.5-7B-Math-Code-v2`
